# **RNN for Text Generation**

In this session, we will train a Recurrent Neural Network (RNN) in PyTorch on the names belonging to several languages. After successful training, the RNN model will predict names belonging to a language that start with an input alphabet letter.

## **Implementation in PyTorch**

Now, we will import all the required libraries.

In [ ]:
!python -m pip install pip --upgrade --user -q --no-warn-script-location
!python -m pip install numpy pandas seaborn matplotlib scipy statsmodels sklearn torch nltk gensim --user -q --no-warn-script-location

import IPython
IPython.Application.instance().kernel.do_shutdown(True)

In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import unicodedata
import string
import torch
import torch.nn as nn
import random
import time
import math
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

The below code snippet will read the dataset.

In [1]:
!wget https://download.pytorch.org/tutorial/data.zip

--2021-10-28 12:52:33--  https://download.pytorch.org/tutorial/data.zip
Resolving download.pytorch.org (download.pytorch.org)... 18.66.78.69, 18.66.78.33, 18.66.78.3, ...
Connecting to download.pytorch.org (download.pytorch.org)|18.66.78.69|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2882130 (2.7M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]   2.75M  3.85MB/s    in 0.7s    

2021-10-28 12:52:35 (3.85 MB/s) - ‘data.zip’ saved [2882130/2882130]



In [ ]:
all_let = string.ascii_letters + " .,;'-"
n_let = len(all_let) + 1

def getFiles(path):
  return glob.glob(path)

# Unicode string to ASCII
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_let
    )

# Read a file and split into lines
def getLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

# Build the cat_lin dictionary, a list of lines per category
cat_lin = {}
all_ctg = []
for filename in getFiles('data/names/*.txt'):
    categ = os.path.splitext(os.path.basename(filename))[0]
    all_ctg.append(categ)
    lines = getLines(filename)
    cat_lin[categ] = lines

n_ctg = len(all_ctg)

In the next step, we will define the module class to generate the names. The module will be a recurrent neural network.

In [ ]:
class NameGeneratorModule(nn.Module):
  def __init__(self, inp_size, hid_size, op_size):
      super(NameGeneratorModule, self).__init__()
      self.hid_size = hid_size

      self.i2h = nn.Linear(n_ctg + inp_size + hid_size, hid_size)
      self.i2o = nn.Linear(n_ctg + inp_size + hid_size, op_size)
      self.o2o = nn.Linear(hid_size + op_size, op_size)
      
      self.dropout = nn.Dropout(0.1)
      self.softmax = nn.LogSoftmax(dim=1)

  def forward(self, category, input, hidden):
      inp_comb = torch.cat((category, input, hidden), 1)
      hidden = self.i2h(inp_comb)
      output = self.i2o(inp_comb)
      op_comb = torch.cat((hidden, output), 1)
      output = self.o2o(op_comb)
      output = self.dropout(output)
      output = self.softmax(output)
      return output, hidden

  def initHidden(self):
      return torch.zeros(1, self.hid_size)

The below functions will be used to pick the random item from a list and a random line from a category

In [ ]:
def randChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randTrainPair():
    category = randChoice(all_ctg)
    line = randChoice(cat_lin[category])
    return category, line

The below functions will convert the data to the compatible format for the RNN module.

In [ ]:
def categ_Tensor(categ):
    li = all_ctg.index(categ)
    tensor = torch.zeros(1, n_ctg)
    tensor[0][li] = 1
    return tensor

def inp_Tensor(line):
    tensor = torch.zeros(len(line), 1, n_let)
    for li in range(len(line)):
        letter = line[li]
        tensor[li][0][all_let.find(letter)] = 1
    return tensor

def tgt_Tensor(line):
    letter_indexes = [all_let.find(line[li]) for li in range(1, len(line))]
    letter_indexes.append(n_let - 1) # EOS
    return torch.LongTensor(letter_indexes)


The below function will create random training examples including category, input and target tensors.

In [ ]:
def rand_train_exp():
    category, line = randTrainPair()
    category_tensor = categ_Tensor(category)
    input_line_tensor = inp_Tensor(line)
    target_line_tensor = tgt_Tensor(line)
    return category_tensor, input_line_tensor, target_line_tensor

The below function will define the loss criteria for the RNN module.

In [ ]:
#Loss
criterion = nn.NLLLoss()
#Learning rate
lr_rate = 0.0005

def train(category_tensor, input_line_tensor, target_line_tensor):
    target_line_tensor.unsqueeze_(-1)
    hidden = model.initHidden()

    model.zero_grad()

    loss = 0

    for i in range(input_line_tensor.size(0)):
        output, hidden = model(category_tensor, input_line_tensor[i], hidden)
        l = criterion(output, target_line_tensor[i])
        loss += l

    loss.backward()

    for p in model.parameters():
        p.data.add_(p.grad.data, alpha=-lr_rate)

    return output, loss.item() / input_line_tensor.size(0)

To show time during training, the bellow function is defined.

In [ ]:
def time_taken(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In the next step, we will define our RNN model.

In [ ]:
model = NameGeneratorModule(n_let, 128, n_let)

We will see the parameters of the defined RNN model.

In [ ]:
print(model)

In [ ]:
def rand_train_exp():
    category, line = randTrainPair()
    category_tensor = categ_Tensor(category)
    input_line_tensor = inp_Tensor(line)
    target_line_tensor = tgt_Tensor(line)
    return category_tensor, input_line_tensor, target_line_tensor

In the next step, the model will be trained in 10,000 epochs.

In [ ]:
epochs = 100000
print_every = 5000
plot_every = 500
all_losses = []
total_loss = 0 # Reset every plot_every iters
#rnn = RNN(n_letters, 128, n_letters)
start = time.time()

for iter in range(1, epochs + 1):
    output, loss = train(*rand_train_exp())
    total_loss += loss

    if iter % print_every == 0:
        print('Time: %s, Epoch: (%d - Total Iterations: %d%%),  Loss: %.4f' % (time_taken(start), iter, iter / epochs * 100, loss))

    if iter % plot_every == 0:
        all_losses.append(total_loss / plot_every)
        total_loss = 0

We will visualize the training loss.

In [ ]:
plt.figure(figsize=(7,7))
plt.title("Loss")
plt.plot(all_losses)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

Finally, we will sample our model to test it on generating the names belonging to languages when given a starting alphabet letter.

In [ ]:
max_length = 20
# Sample from a category and starting letter
def sample_model(category, start_letter='A'):
    with torch.no_grad():  # no need to track history in sampling
        category_tensor = categ_Tensor(category)
        input = inp_Tensor(start_letter)
        hidden = model.initHidden()

        output_name = start_letter

        for i in range(max_length):
            output, hidden = model(category_tensor, input[0], hidden)
            topv, topi = output.topk(1)
            topi = topi[0][0]
            if topi == n_let - 1:
                break
            else:
                letter = all_let[topi]
                output_name += letter
            input = inp_Tensor(letter)

        return output_name

# Get multiple samples from one category and multiple starting letters
def sample_names(category, start_letters='XYZ'):
    for start_letter in start_letters:
        print(sample_model(category, start_letter))

Now, we will check the sampled model to generate names when given a language and the starting alphabet letter.

In [ ]:
print("Italian:-")
sample_names('Italian', 'BPRT')
print("\nKorean:-")
sample_names('Korean', 'CMRS')
print("\nRussian:-")
sample_names('Russian', 'AJLN')
print("\nVietnamese:-")
sample_names('Vietnamese', 'LMT')

# **Related Articles:**

> * [Text Generation using RNN](https://analyticsindiamag.com/recurrent-neural-network-in-pytorch-for-text-generation/)

> * [SVD in Recommender System](https://analyticsindiamag.com/singular-value-decomposition-svd-application-recommender-system/)

> * [TF-IDF from Scratch in Python](https://analyticsindiamag.com/hands-on-implementation-of-tf-idf-from-scratch-in-python/)

> * [Continuous Bag of Words](https://analyticsindiamag.com/the-continuous-bag-of-words-cbow-model-in-nlp-hands-on-implementation-with-codes/)

> * [NLP Case Study of Documents Similarity](https://analyticsindiamag.com/nlp-case-study-identify/)

> * [Review Classification](https://analyticsindiamag.com/step-by-step-guide-to-reviews-classification-using-svc-naive-bayes-random-forest/)

